In [ ]:
import os
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from openai import AsyncOpenAI
import asyncio

openai_api_key = "Enter You OpenAI key"
if not openai_api_key:
    print('Warning: OPENAI_API_KEY not set. Set it before running LLM calls.')

# Initialize the OpenAI client with the API key
client = AsyncOpenAI(api_key=openai_api_key)


In [41]:
# ==== Константы и загрузка данных ====
CLIENTS_PATH = "case 1/clients.csv"
clients_df = pd.read_csv(CLIENTS_PATH)

In [42]:
def load_transactions(client_code):
    path = f"client_{client_code}_transactions_3m.csv"
    if os.path.exists(path):
        return pd.read_csv(path)
    return pd.DataFrame(columns=['date', 'category', 'amount', 'currency', 'client_code', 'status', 'city', 'name', 'product'])

def load_transfers(client_code):
    path = f"client_{client_code}_transfers_3m.csv"
    if os.path.exists(path):
        return pd.read_csv(path)
    return pd.DataFrame(columns=['date', 'type', 'direction', 'amount', 'currency', 'client_code', 'status', 'city', 'name', 'product'])

In [43]:
# ====================================================================================
# ШАГ 1: КЛАСТЕРИЗАЦИЯ КЛИЕНТОВ (ML)
# ====================================================================================

def create_features_for_clustering(profile, tx, tr):
    """Собирает числовые признаки (фичи) для одного клиента."""
    features = {}
    
    # Демография и баланс
    features['age'] = profile['age']
    features['avg_monthly_balance_KZT'] = profile['avg_monthly_balance_KZT']
    
    # Признаки из транзакций
    if not tx.empty:
        features['total_spend_3m'] = tx['amount'].sum()
        features['tx_count_3m'] = len(tx)
        features['avg_check'] = tx['amount'].mean()
        online_spend = tx[tx["category"].isin(["Едим дома", "Смотрим дома", "Играем дома"])]["amount"].sum()
        features['online_spend_ratio'] = online_spend / features['total_spend_3m'] if features['total_spend_3m'] > 0 else 0
    else:
        features['total_spend_3m'] = 0
        features['tx_count_3m'] = 0
        features['avg_check'] = 0
        features['online_spend_ratio'] = 0
        
    # Признаки из переводов
    if not tr.empty:
        inflow = tr[tr['direction'] == 'in']['amount'].sum()
        outflow = tr[tr['direction'] == 'out']['amount'].sum()
        features['inflow_3m'] = inflow
        features['outflow_3m'] = outflow
        features['savings_rate'] = (inflow - outflow) / inflow if inflow > 0 else 0
    else:
        features['inflow_3m'] = 0
        features['outflow_3m'] = 0
        features['savings_rate'] = 0
        
    return features

In [44]:
# --- Подготовка данных для кластеризации ---
all_features = []
for _, profile in clients_df.iterrows():
    client_code = profile['client_code']
    tx = load_transactions(client_code)
    tr = load_transfers(client_code)
    client_features = create_features_for_clustering(profile, tx, tr)
    client_features['client_code'] = client_code
    all_features.append(client_features)

features_df = pd.DataFrame(all_features).set_index('client_code')

# --- Масштабирование и обучение модели KMeans ---
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features_df)

kmeans = KMeans(n_clusters=3, random_state=42, n_init='auto')
clients_df['cluster_id'] = kmeans.fit_predict(scaled_features)


# Интерпретация кластеров (гипотеза на основе типовых данных):
# 0 - "Молодые и цифровые" (низкий баланс, высокий онлайн-коэффициент)
# 1 - "Состоятельные" (высокий баланс и траты, возраст)
# 2 - "Стабильные и экономные" (средний возраст, есть накопления)

In [45]:
# ====================================================================================
# ШАГ 2: ВНЕДРЕНИЕ ПРАВИЛ ДОСТУПНОСТИ И СКЛОННОСТИ
# ====================================================================================

def check_eligibility(profile, product_name):
    """Проверяет, доступен ли продукт клиенту. Возвращает True/False."""
    balance = profile['avg_monthly_balance_KZT']
    status = profile['status']
    age = profile['age']

    if product_name == "Кредит наличными":
        return status != 'Студент' and age >= 21
    if product_name == "Премиальная карта":
        return balance >= 1_000_000 or status == 'Премиальный клиент'
    if product_name == "Инвестиции":
        return age >= 18
    if product_name == "Депозит сберегательный":
        # Условное правило: предлагаем "заморозить" деньги тем, у кого их достаточно
        return balance > 500_000 
    
    return True # Все остальные продукты доступны по умолчанию

def calculate_propensity(profile, tx, tr):
    """Оценивает склонность (интерес) клиента к продуктам. Возвращает словарь {продукт: балл от 0 до 1}."""
    propensities = {
        "Карта для путешествий": 0.5, "Премиальная карта": 0.5, "Кредитная карта": 0.5,
        "Обмен валют": 0.5, "Кредит наличными": 0.5, "Депозит мультивалютный": 0.5,
        "Депозит сберегательный": 0.5, "Депозит накопительный": 0.5, "Инвестиции": 0.5,
        "Золотые слитки": 0.5
    }

    # Сигналы из транзакций
    if not tx.empty:
        categories = set(tx['category'])
        if "Путешествия" in categories or "Отели" in categories:
            propensities["Карта для путешествий"] = 1.0
        # Новое: если клиент часто тратит в валюте, повысить склонность к обмену валют
        if (tx['currency'] != 'KZT').sum() > 2:
            propensities["Обмен валют"] = 1.0
        # Новое: если город Алматы или Астана, повысить склонность к премиальным продуктам
        if tx['city'].isin(['Алматы', 'Астана']).any():
            propensities["Премиальная карта"] = min(1.0, propensities["Премиальная карта"] + 0.3)

    # Сигналы из переводов
    if not tr.empty:
        types = set(tr['type'])
        if 'cc_repayment_out' in types or 'installment_payment_out' in types:
            propensities["Кредитная карта"] = 1.0
        if 'invest_in' in types or 'invest_out' in types:
            propensities["Инвестиции"] = 1.0
        if 'fx_buy' in types or 'fx_sell' in types:
            propensities["Обмен валют"] = 1.0
        inflow = tr[tr['direction'] == 'in']['amount'].sum()
        outflow = tr[tr['direction'] == 'out']['amount'].sum()
        if outflow > inflow * 1.2:
            propensities["Кредит наличными"] = 0.9
        # Новое: если есть переводы с продуктом "Депозит", повысить склонность к депозитам
        if tr['product'].str.contains('депозит', case=False, na=False).any():
            propensities["Депозит сберегательный"] = min(1.0, propensities["Депозит сберегательный"] + 0.3)

    # Сигналы из профиля
    if profile['status'] in ['Премиальный клиент', 'Зарплатный клиент'] and profile['age'] in range(25, 45):
        propensities["Инвестиции"] = min(1.0, propensities["Инвестиции"] + 0.4)

    return propensities


In [46]:
# ====================================================================================
# ШАГ 3: ОБНОВЛЕНИЕ ОСНОВНОЙ ЛОГИКИ
# ====================================================================================

# Функция расчета выгоды остается прежней (с исправлениями из прошлого шага)
def calculate_benefits(profile, tx, tr):
    benefits = {}
    balance = profile["avg_monthly_balance_KZT"]

    if not tx.empty:
        total_spend = tx["amount"].sum()
        travel_spend = tx[tx["category"].isin(["Путешествия", "Такси", "Отели"])]["amount"].sum()
        benefits["Карта для путешествий"] = 0.04 * travel_spend
        extra_categories = ["Кафе и рестораны", "Ювелирные украшения", "Косметика и Парфюмерия"]
        extra_spend = tx[tx["category"].isin(extra_categories)]["amount"].sum()
        base_spend = total_spend - extra_spend
        tier_rate = 0.04 if balance >= 6_000_000 else (0.03 if balance >= 1_000_000 else 0.02)
        total_cashback = (base_spend * tier_rate) + (extra_spend * 0.04)
        benefits["Премиальная карта"] = min(300_000, total_cashback)
        cats = tx.groupby("category")["amount"].sum().sort_values(ascending=False)
        top3_spend = cats.head(3).sum()
        online_spend = tx[tx["category"].isin(["Едим дома", "Смотрим дома", "Играем дома"])]["amount"].sum()
        benefits["Кредитная карта"] = 0.1 * top3_spend + 0.1 * online_spend
    else:
        benefits.update({"Карта для путешествий": 0, "Премиальная карта": 0, "Кредитная карта": 0})

    fx_ops_amount = tr[tr["type"].isin(["fx_buy", "fx_sell"])]["amount"].sum() if not tr.empty else 0
    benefits["Обмен валют"] = 0.01 * fx_ops_amount

    inflow = tr[tr["direction"] == "in"]["amount"].sum() if not tr.empty else 0
    outflow = tr[tr["direction"] == "out"]["amount"].sum() if not tr.empty else 0
    benefits["Кредит наличными"] = (outflow - inflow) * 0.02 if outflow > inflow * 1.2 else 0

    if balance > 100_000:
        benefits["Депозит мультивалютный"] = balance * 0.145 / 4
        benefits["Депозит сберегательный"] = balance * 0.165 / 4
        benefits["Депозит накопительный"] = balance * 0.155 / 4
    else:
        benefits.update({"Депозит мультивалютный": 0, "Депозит сберегательный": 0, "Депозит накопительный": 0})
        
    benefits["Инвестиции"] = balance * 0.01 if balance > 50_000 else 0
    benefits["Золотые слитки"] = balance * 0.005 if balance > 200_000 else 0

    return benefits



In [47]:
async def generate_push_with_tov(profile, product, benefits, tx, tr, cluster_id):
    """Generate a push notification using OpenAI ChatCompletion following TOV and constraints."""
    name = profile.get('name', 'клиент')

    def fnum(n):
        try:
            n = float(n)
            if n.is_integer():
                return f'{int(n):,}'.replace(',', ' ')
            # use comma as decimal separator and spaces for thousands
            return f'{n:,.2f}'.replace(',', ' ').replace('.', ',')
        except Exception:
            return str(n)

    tov_system = (
        'Вы — копирайтер для push-уведомлений банка. Тон: дружелюбный, простой, обращение на "вы" с маленькой буквы. ',
        'Важная информация — в начале. Без воды, лёгкий юмор допустим, 0-1 эмодзи. Без КАПС, максимум один восклицательный знак. ',
        'Длина: 180-220 символов. Формат чисел: пробелы для разрядов, запятая — десятичный разделитель. CTA — однословный глагол (Открыть/Посмотреть/Настроить).'
    )
    tov_system = ''.join(tov_system)

    cluster_description = {0: 'Молодые и цифровые', 1: 'Состоятельные', 2: 'Стабильные и экономные'}

    # Top categories summary
    if not tx.empty:
        top_cats = tx.groupby('category')['amount'].sum().nlargest(3)
        top_cats_str = ', '.join(top_cats.index.tolist())
    else:
        top_cats_str = 'N/A'

    estimated_benefit = fnum(benefits.get(product, 0))

    user_prompt = (
        f"Клиент: имя={name}, возраст={profile.get('age')}, статус={profile.get('status')}, баланс={fnum(profile.get('avg_monthly_balance_KZT',0))} ₸. "
        f"Кластер: {cluster_description.get(cluster_id)}. Рекомендуемый продукт: {product}. "
        f"Оценочная выгода: {estimated_benefit} ₸. Топ категорий: {top_cats_str}. "
        'Напишите один короткий пуш (180-220 символов) по указанному тону и добавьте CTA одним словом. Используйте 0-1 эмодзи по смыслу.'
    )
    user_prompt = ''.join(user_prompt)

    messages = [
        {'role': 'system', 'content': tov_system},
        {'role': 'user', 'content': user_prompt}
    ]

    # If API key is missing, return safe template
    if not openai_api_key:
        return f'{name}, у нас есть персональное предложение по {product}. Посмотрите детали в приложении.'

    last_err = None
    for attempt in range(3):
        try:
            resp = await client.chat.completions.create(model='gpt-4', messages=messages, max_tokens=180, temperature=0.7)
            text = resp.choices[0].message.content.strip()
            # Ensure length constraint roughly (fallback if model returns too long)
            if len(text) > 240:
                text = text[:240].rsplit(' ', 1)[0] + '.'
            return text
        except Exception as e:
            last_err = e
            await asyncio.sleep(1 + attempt * 2)
    print(f'Error generating push notification with OpenAI API: {last_err}')
    return f'{name}, для вас есть персональное предложение. Узнайте подробности в приложении.'

# ====================================================================================
# ШАГ 4: ГЕНЕРАЦИЯ ФИНАЛЬНОГО CSV
# ====================================================================================

final_recommendations = []

for _, profile in clients_df.iterrows():
    client_code = profile['client_code']
    tx = load_transactions(client_code)
    tr = load_transfers(client_code)

    # Check eligibility and calculate propensity
    propensities = calculate_propensity(profile, tx, tr)
    eligible_products = {product: score for product, score in propensities.items() if check_eligibility(profile, product)}

    # Select the best product based on propensity score
    if eligible_products:
        product = max(eligible_products, key=eligible_products.get)
    else:
        product = 'Нет доступных продуктов'

    # Calculate benefits
    benefits = calculate_benefits(profile, tx, tr)

    # Generate push notification
    cluster_id = profile['cluster_id']
    push_text = await generate_push_with_tov(profile, product, benefits, tx, tr, cluster_id)

    # Append final recommendation (only required columns)
    final_recommendations.append({
        'client_code': int(client_code),
        'recommended_product': product,
        'push_notification': push_text
    })

# Convert to DataFrame and save to CSV (only 3 columns)
final_recommendations_df = pd.DataFrame(final_recommendations)
final_recommendations_df.to_csv('final_recommendations.csv', index=False)

print("Финальный файл 'final_recommendations.csv' успешно создан.")

Финальный файл 'final_recommendations.csv' успешно создан.
